In [2]:
# Import packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [3]:
# Import and clean up dataset
df = pd.read_csv('data/electric_vehicles_spec_2025.csv')
df = df.dropna()
df.head()

,brand,model,top_speed_kmh,battery_capacity_kWh,battery_type,number_of_cells,torque_nm,efficiency_wh_per_km,range_km,acceleration_0_100_s,...,towing_capacity_kg,cargo_volume_l,seats,drivetrain,segment,length_mm,width_mm,height_mm,car_body_type,source_url
0,Abarth,500e Convertible,155,37.8,Lithium-ion,192.0,235.0,156,225,7.0,...,0.0,185,4,FWD,B - Compact,3673,1683,1518,Hatchback,https://ev-database.org/car/1904/Abarth-500e-C...
1,Abarth,500e Hatchback,155,37.8,Lithium-ion,192.0,235.0,149,225,7.0,...,0.0,185,4,FWD,B - Compact,3673,1683,1518,Hatchback,https://ev-database.org/car/1903/Abarth-500e-H...
2,Abarth,600e Scorpionissima,200,50.8,Lithium-ion,102.0,345.0,158,280,5.9,...,0.0,360,5,FWD,JB - Compact,4187,1779,1557,SUV,https://ev-database.org/car/3057/Abarth-600e-S...
3,Abarth,600e Turismo,200,50.8,Lithium-ion,102.0,345.0,158,280,6.2,...,0.0,360,5,FWD,JB - Compact,4187,1779,1557,SUV,https://ev-database.org/car/3056/Abarth-600e-T...
6,Alfa,Romeo Junior Elettrica 54 kWh,150,50.8,Lithium-ion,102.0,260.0,128,320,9.0,...,0.0,400,5,FWD,JB - Compact,4173,1781,1532,SUV,https://ev-database.org/car/2184/Alfa-Romeo-Ju...


In [4]:
# Check for missing values
df.isnull().sum()

brand                        0
model                        0
top_speed_kmh                0
battery_capacity_kWh         0
battery_type                 0
number_of_cells              0
torque_nm                    0
efficiency_wh_per_km         0
range_km                     0
acceleration_0_100_s         0
fast_charging_power_kw_dc    0
fast_charge_port             0
towing_capacity_kg           0
cargo_volume_l               0
seats                        0
drivetrain                   0
segment                      0
length_mm                    0
width_mm                     0
height_mm                    0
car_body_type                0
source_url                   0
dtype: int64

Using the dataset from Kaggle about electric vehicles - https://www.kaggle.com/datasets/urvishahir/electric-vehicle-specifications-dataset-2025

Description :

This dataset provides a comprehensive collection of specifications and performance metrics for modern electric vehicles (EVs). It is designed to support researchers, analysts, students, and developers working on data science, machine learning, automotive market research, sustainability studies, or electric vehicle adoption analysis.

Each row in the dataset represents a specific electric vehicle model with a rich set of attributes covering:


        Brand and Model: Manufacturer and specific nameplate of the EV.
        Car Body Type: Classification such as hatchback, SUV, sedan, etc.
        Segment: Vehicle segment (e.g., compact, midsize, executive).

        Battery Capacity (kWh): The gross energy capacity of the battery.
        Number of Cells and Battery Type: Technical battery information, where available.
        Efficiency (Wh/km): Power consumption rate of the vehicle.
        Range (km): Estimated driving range on a full charge.

        Fast Charging Power (kW): Maximum supported DC fast-charging power.
        Fast Charge Port Type: Connector standard (e.g., CCS, CHAdeMO).

        Top Speed (km/h): Maximum speed of the vehicle.
        0–100 km/h Acceleration (s): Time to reach 100 km/h from a standstill.
        Torque (Nm): Maximum torque output, where available.

        Towing Capacity (kg): Ability to tow loads, provided where applicable.
        Cargo Volume (L): Luggage space, sometimes approximate or expressed in alternative units.
        Seats: Total seating capacity.
        
        Length, Width, Height (mm): Physical footprint of the vehicle.
        Drivetrain: Powertrain configuration (e.g., AWD, RWD, FWD).
        Source URL: Reference link for each car (used in scraping).

Question 1: Is there a relationship between the technical qualities of the batteries? 
- Unsupervised
- No latent variable
- No clear truth variable
- Perform clustering
- Perform other unsupervised modelingQuestion: What other factors are closely related to efficiency? What makes an electric car the most efficient?

In [9]:
# Could cluster car brands together based on technical specs: battery size, top speeds, number of cells, etc...
scaler = StandardScaler()
technical_features = df[['top_speed_kmh', 'battery_capacity_kWh', 'number_of_cells', 
                      'torque_nm', 'efficiency_wh_per_km', 'range_km', 'acceleration_0_100_s']].values


technical_features_scaled = scaler.fit_transform(technical_features)

In [10]:
technical_features_scaled

array([[-0.85245959, -1.86180745, -0.21884879, ..., -0.18548153,
        -1.64314963,  0.02101304],
       [-0.85245959, -1.86180745, -0.21884879, ..., -0.40922165,
        -1.64314963,  0.02101304],
       [ 0.4596964 , -1.18704578, -0.34829482, ..., -0.12155578,
        -1.10591629, -0.3603737 ],
       ...,
       [ 0.4596964 ,  1.05523918, -0.3367885 , ..., -0.31333303,
         1.09185647,  0.09035609],
       [ 0.4596964 ,  1.05523918, -0.3367885 , ..., -0.08959291,
         0.84765949, -1.08847567],
       [ 0.4596964 ,  1.05523918, -0.3367885 , ...,  0.00629572,
         0.84765949, -1.08847567]])

Question 2: 
- What other factors are closely related to efficiency? What makes an electric car the most efficient?

Question 3:

Predictive Modeling

Build a predictive model: Given variables like battery_capacity_kWh, body type, segment, drivetrain, brand, dimensions — can you predict range_km?

Or: Predict fast_charging_power_kW given other specs (battery size, brand, segment) — which vehicles support high charging?

Cluster analysis: Are there natural clusters of EVs (e.g., economy commuter, high-performance, luxury long-range) based on specs?

Feature engineering: Create derived metrics like “range per kWh”, “Wh per km per kg volume”, etc.— which features are most predictive of being in “premium” vs “economy” class?